In [ ]:
new_virtual = {"internal": ["internal-23q2-1e49", 75], "dmc": ["dmc-23q2-d87c", 70], "public": ["public-23q2-19de", 70]}
old_virtual = {"internal": "internal-22q2-097a", "dmc": "dmc-22q2-5e51", "public": "public-22q2-de04"}

In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()

In [ ]:
# portal = "internal"
from depmap_omics_upload import tracker as track
import pandas as pd

mytracker = track.SampleTracker()
pr_table = mytracker.add_model_cols_to_prtable(["ModelID"])
blacklist = set(['ACH-001956', 'ACH-000116', 'ACH-001957', 'ACH-001955'])

def appendLegacyCN(portal):
    new_mat = tc.get(name=new_virtual[portal][0], version=new_virtual[portal][1], file='OmicsCNGene')
    old_mat = tc.get(name=old_virtual[portal], file='CCLE_gene_cn')
    legacy_segments = tc.get(name='copy-number-5f61', version=5, file='legacy_segments')
    default_model_table = tc.get(name=new_virtual[portal][0], version=new_virtual[portal][1], file='OmicsDefaultModelProfiles')
    default_mc_table = tc.get(name=new_virtual[portal][0], version=new_virtual[portal][1], file='OmicsDefaultModelConditionProfiles')
    OmicsProfiles = tc.get(name=new_virtual[portal][0], version=new_virtual[portal][1], file='OmicsProfiles')
    
    legacy_models = set(legacy_segments.DepMap_ID)

    new_models = set(new_mat.index)
    old_models = set(old_mat.index)

    missing_in_new = old_models - new_models
    missing_in_old = new_models - old_models

    to_append = [l for l in missing_in_new if (l in legacy_models and l not in blacklist)]
    print("appending " + str(len(to_append)) + " rows for " + portal)
    
    rename_model_to_pr = dict()
    rename_mc_to_pr = dict()
    prs_to_append = []
    for l in to_append:
        if len(pr_table[(pr_table.ModelID == l) & (pr_table.ProfileSource == "taiga")]) == 1:
            pr = pr_table[(pr_table.ModelID == l) & (pr_table.ProfileSource == "taiga")].index[0]
            mc = pr_table.loc[pr, "ModelCondition"]
            prs_to_append.append(pr)
            rename_model_to_pr[l] = pr
            rename_mc_to_pr[mc] = pr

    to_append_df = old_mat[old_mat.index.isin(to_append)]
    
    # append legacy to profile table
    profile_table_to_append = pr_table.loc[prs_to_append, ["ModelCondition", "ModelID", "Datatype", "Baits"]].reset_index()
    profile_table_to_append = profile_table_to_append.rename(columns={"Baits": "WESKit"})
    profile_table_to_append = profile_table_to_append[profile_table_to_append["Datatype"].isin(["rna", "wgs", "wes", "SNParray"])]
    OmicsProfiles = pd.concat([OmicsProfiles, profile_table_to_append], ignore_index=True)
    OmicsProfiles = OmicsProfiles.rename(columns={"ModelCondition": "ModelConditionID"})
    OmicsProfiles.loc[
        OmicsProfiles[OmicsProfiles["Datatype"].isin(["rna", "wgs", "SNParray"])].index.tolist(),
        "WESKit",
    ] = ""
    OmicsProfiles.to_csv("../output/22Q4/omics_profiles_with_legacy.csv", index=False)

    # append and save cn matrix
    full_mat = new_mat.append(to_append_df)
    full_mat.to_csv("../output/22Q4/gene_cn_with_legacy.csv")
    
    # appending to default tables
    app_mc = pd.DataFrame.from_dict(rename_mc_to_pr, orient='index').reset_index()
    app_mc.columns=['ModelConditionID', "ProfileID"]
    app_mc["ProfileType"] = "CN only"
    full_mc_table = pd.concat([default_mc_table, app_mc], ignore_index=True)
    full_mc_table.to_csv("../output/22Q4/mc_table_with_legacy.csv", index=False)
    
    app_model = pd.DataFrame.from_dict(rename_model_to_pr, orient='index').reset_index()
    app_model.columns=['ModelID', "ProfileID"]
    app_model["ProfileType"] = "CN only"
    full_model_table = pd.concat([default_model_table, app_model], ignore_index=True)
    full_model_table.to_csv("../output/22Q4/model_table_with_legacy.csv", index=False) 
    
    tc.update_dataset(
        changes_description="append legacy",
        dataset_id=new_virtual[portal][0],
        upload_files=[
            {
                "path": "../output/22Q4/gene_cn_with_legacy.csv",
                "name": "OmicsCNGene",
                "format": "NumericMatrixCSV",
                "encoding": "utf-8",
            },
            {
                "path": "../output/22Q4/mc_table_with_legacy.csv",
                "name": "OmicsDefaultModelConditionProfiles",
                "format": "TableCSV",
                "encoding": "utf-8",
            },
            {
                "path": "../output/22Q4/model_table_with_legacy.csv",
                "name": "OmicsDefaultModelProfiles",
                "format": "TableCSV",
                "encoding": "utf-8",
            },
            {
                "path": "../output/22Q4/omics_profiles_with_legacy.csv",
                "name": "OmicsProfiles",
                "format": "TableCSV",
                "encoding": "utf-8",
            },
        ],
        add_all_existing_files=True,
    )
    
    return OmicsProfiles, full_mc_table, full_model_table, full_mat

In [ ]:
new_files = dict()
for p in new_virtual.keys():
    profile_table, full_mc_table, full_model_table, full_mat = appendLegacyCN(p)
    new_files[p] = [profile_table, full_mc_table, full_model_table, full_mat]

In [ ]:
new_mat = tc.get(name=new_virtual["public"][0], version=new_virtual["public"][1], file='OmicsCNGene')
old_mat = tc.get(name=old_virtual["internal"], file='CCLE_gene_cn')
legacy_segments = tc.get(name='copy-number-5f61', version=5, file='legacy_segments')

legacy_models = set(legacy_segments.DepMap_ID)

new_models = set(new_mat.index)
old_models = set(old_mat.index)

missing_in_new = old_models - new_models
missing_in_old = new_models - old_models

to_append = [l for l in missing_in_new if (l in legacy_models and l not in blacklist)]

In [ ]:
len(to_append)

In [ ]:
from depmap_omics_upload import tracker as track

mytracker = track.SampleTracker()
pr_table = mytracker.add_model_cols_to_prtable(["ModelID"])

In [ ]:
# do the sam on seg level for latest:
new_mat = tc.get(name="cn-latest-d8d4", version=38, file='merged_gene_cn_profile')
old_mat = tc.get(name="internal-22q2-097a", file='CCLE_gene_cn')

to_append = [i for i in to_append if i not in ['ACH-001956', 'ACH-001955', 'ACH-001957', 'ACH-000116']]

rename_model_to_pr = dict()
rename_mc_to_pr = dict()
prs_to_append = []

for l in to_append:
    if len(pr_table[(pr_table.ModelID == l) & (pr_table.ProfileSource == "taiga")]) == 1:
        pr = pr_table[(pr_table.ModelID == l) & (pr_table.ProfileSource == "taiga")].index[0]
        mc = pr_table.loc[pr, "ModelCondition"]
        prs_to_append.append(pr)
        rename_model_to_pr[l] = pr
        
to_append_df = old_mat[old_mat.index.isin(to_append)]

full_mat = new_mat.append(to_append_df)
full_mat = full_mat.rename(index=rename_model_to_pr)
full_mat = full_mat[full_mat.index.str.startswith("PR-")]

full_mat.to_csv("../output/22Q4/gene_cn_with_legacy_pr.csv")


In [ ]:
tc.update_dataset(
    changes_description="append legacy",
    dataset_id="cn-latest-d8d4",
    upload_files=[
        {
            "path": "../output/22Q4/gene_cn_with_legacy_pr.csv",
            "name": "merged_gene_cn_profile",
            "format": "NumericMatrixCSV",
            "encoding": "utf-8",
        },
    ],
    add_all_existing_files=True,
)